# Travaux pratiques sur le module 1

Ce premier TP vous permettra de manipuler les concepts et méthodes vus dans la vidéo du module 1.

Vous allez découvrir le jeu de données `ozone`, qui contient des mesures métérologiques, et vous garderez cet exemple d'application dans les TPs suivants. Plus précisément, vous visualiserez les données manquantes, interpréterez les graphiques et analyserez les performances des deux méthodes naïves pour le traitement des valeurs manquantes, la suppression des individus incomplets et l'imputation par la moyenne.

Les réponses aux questions ne demandent pas de coder. Le seul prérequis est de savoir lire le code R.


# Installation et chargement des librairies

In [ ]:
# Attention, l'installation des librairies peut prendre du temps (8 minutes)
install.packages(c('missMDA', 'VIM'), dependencies = TRUE)

In [ ]:
library(missMDA)
library(VIM)

# Chargement des données

Dans la suite, vous utiliserez le jeu de données `ozone` du package `missMDA`. Ce jeu de données est composé de 112 mesures atmosphériques prises au niveau du sol à intervalles quotidiens, à Rennes (France) pendant l'été 2001, de début juin à fin septembre.

In [ ]:
data(ozone) # chargement des données
head(ozone) # affichage des 6 premières lignes
tail(ozone) # affichage des 6 dernières lignes

## Question 1

Comme d'habiture en statistique, chaque ligne représente un ensemble de mesures simultanées, en l'occurence une date spécifique : c'est visible dans l'**index** (`20010601, 20010602`: 1er, 2 juin 2001). Avec le code ci-dessous, combien de lignes y a-t-il dans le jeu de données ? Y a-t-il des jours manquants ?

In [ ]:
n.rows <- dim(ozone)[1]
print(n.rows)

n.days.theoretical <- 30 + 31 + 31 + 30 # juin, juillet, août, septembre
print(n.days.theoretical)

### Correction

In [ ]:
print(paste("ozone contient", n.rows, "lignes"))
print(paste("Nombre de jours théoriques: ", n.days.theoretical))

Il manque 10 jours de données entre le 1er juin et le 30 septembre.

# Visualisation des données manquantes

Chaque colonne représente une variable différente, c'est-à-dire ici une grandeur météorologique mesurée : concentration en ozone, température, nébulosité, direction du vent, conditions de pluie...

`ozone` contient 13 variables dont des variables catégorielles, mais pour cette étude, vous vous concentrerez sur les 4 premières variables numériques :
* `maxO3`: la concentration maximale en ozone sur la journée,
* `T9`: la température à 9h,
* `T12`: la température à 12h,
* `T15`: la température à 15h.

In [ ]:
ozone_numeric <- ozone[, 1:4] # sélection des 4 premières variables (colonnes)

In [ ]:
head(ozone_numeric)

## Question 2

Pouvez-vous d'ores et déjà observer des valeurs manquantes dans l'affichage précédent ? Comment sont-elles encodées ?

### Correction

Les valeurs manquantes sont encodées comme `NA`et apparaissent comme tels dans les affichages ci-dessus.

## Question 3a

Vous pouvez utiliser le package `VIM` (Vizualisation and Imputation of Missing values) pour mieux comprendre la répartition des valeurs manquantes. Exécutez la cellule ci-dessous : comment pouvez-vous interpréter le graphique de gauche ? Y a-t-il une tendance qui se dégage ? Que peut-on faire comme hypothèse sur l'occurence de valeurs manquantes sur une journée ?

In [ ]:
a <- aggr(ozone_numeric, plot = FALSE)
par(bg = "white") # définit le fond en blanc
plot(a, numbers = TRUE, prop = FALSE)

### Correction

Le graphique de gauche est un diagramme en barres représentant le nombre de valeurs manquantes dans chaque variable. Il montre que plus on avance dans la journée, plus il y a de valeurs manquantes dans la température. On pourrait alors penser qu'il y a un phénomène de dépendance, et que si la température à 9h est manquante, alors elle le sera forcément aux heures suivantes. Le graphique de gauche ne permet pas de vérifier cette hypothèse.

## Question 3b

Comment pouvez-vous interpréter le graphique de droite ? Permet-il de confirmer ou d'informer votre hypothèse ?

### Correction

Le graphique de droite fait la liste de tous les *patterns* de données manquantes (ou motifs de manque), c'est-à-dire toutes les combinaisons possibles de variables manquantes pour une seule ligne, et les classes par fréquence.

On observe ainsi que le cas le plus fréquent (65 lignes) est que toutes les variables soient observées, et le deuxième cas le plus fréquent (13 lignes) est que `T15` seule soit manquante.

Notre hypothèse précédente est-elle vérifiée ? Non : il arrive que `T9`soit manquante sans que `T12` ou `T15` le soient (sur 6 lignes, `T9` est manquante et `T12`, `T15` observées).

## Question 4

Refaites l'exercice avec `ozone`tout entier. Que pouvez-vous observer ? Les tendances précédentes restent-elles vraies ? Y a-t-il des lignes avec toutes les variables manquantes ? Pourquoi ?

### Correction

In [ ]:
a <- aggr(ozone, plot = FALSE)
par(bg = "white") # définit le fond en blanc
plot(a, numbers = TRUE, prop = FALSE)

Grâce au graphique de gauche, on observe, sur la nébulosité et la direction du vent, le même phénomène que sur la température : plus on avance dans la journée, plus il y a de valeurs manquantes. Le graphique de droite montre néanmoins, de même, que la nébulosité peut être manquante à 9h sans qu'elle le soit après.

Il n'y a aucune ligne avec toutes les variables manquantes... vraiment ? Rappelez-vous la question 1 : il manque 10 lignes dans le jeu de données ! Les lignes totalement vides ont donc été préalablement retirées du jeu de données, car elles ne contiennent aucune information.

# Mécanismes de données manquantes

## Question 5

Pour comprendre pourquoi le jeu de données est incomplet, il faut bien souvent discuter avec les experts du domaine d'application et/ou ceux qui ont collecté les données. Parfois, ce n'est pas possible et des hypothèses doivent être faites quant au type de de données manquantes.

Avec le package `VIM`, on peut représenter les variables deux à deux, et leur manque pour essayer d'en tirer des conclusions sur le mécanisme de données manquantes. Dans le graphique ci-dessous, en abscisse, le boxplot rouge correspond à la distribution de toutes les valeurs de la variable `maxO3` lorsque la variable `T9` est manquante. Le boxplot rouge correspond à la distribution de toutes les valeurs de la variable `maxO3` lorsque la variable `T9` est manquante.

Avec les boxplots du graphique ci-dessous, pouvez-vous faire une hypothèse sur le mécanisme de données manquantes ?

In [ ]:
x <- ozone[, c("maxO3", "T9")]
marginplot(x)

### Correction

Le mécanisme peut être considéré comme étant Missing At Random (MAR) ou Missing Not At Random (MNAR), car les distributions de la variable `maxO3` selon que la variable `T9` ait observée ou non diffèrent. On peut donc supposer que le manque de la variable `maxO3` est relié aux valeurs (observées ou manquantes) de la variable `T9`.

# Méthodes naïves pour traiter les `NA`

Dans la suite du TP, vous comparerez deux méthodes naïves pour le traitement des valeurs manquantes, mentionnées dans la vidéo du module 1 :
- **La suppression des individus incomplets:** cette stratégie consiste à supprimer toutes les lignes incomplètes (une ligne est dite incomplète dès qu'elle contient au moins une valeur manquante).
- **L'imputation par la moyenne:** cette méthode consiste pour chaque variable, à prédire les valeurs manquantes par la moyenne de la variable.

Vous comparez ces méthodes sur leur estimation de la moyenne empirique et de la variance empirique de chaque variable.

Dans la suite du TP, vous restreindrez votre analyse sur les variables continues.

In [ ]:
ozone <- ozone[, 1:11]

## Question 6

En utilisant les sorties du code ci-dessous, pensez-vous que la méthode de supression des individus incomplets est pertinente ? Pourquoi ?

In [ ]:
ozone.full <- na.omit(ozone) # jeu de données avec seulement les lignes complètes

In [ ]:
n.full.rows <- dim(ozone.full)[1]
print(n.full.rows)

In [ ]:
print(apply(ozone.full, 2, mean, na.rm = TRUE))

In [ ]:
print(apply(ozone.full, 2, var, na.rm = TRUE))

### Correction

Il n'y a que 34 lignes complètes dans le jeu de données. Il y a donc beaucoup de perte d'information.

## Question 7

La fonction `impute.mean` ci-dessous implémente la stratégie d'imputation par la moyenne. Comparez cette méthode à la suppression des lignes incomplètes.

In [ ]:
impute.mean <- function(data) { #fonction d'imputation par la moyenne
  data.imputed <- data
  for (j in 1:11) {
    data.imputed[which(is.na(data[, j])), j] <- mean(data[, j], na.rm = TRUE)
  }
  return(data.imputed)
}

In [ ]:
ozone.mean <- impute.mean(ozone)

In [ ]:
print(apply(ozone.mean, 2, mean))

In [ ]:
print(apply(ozone.mean, 2, var))

### Correction

On observe des valeurs différentes pour les deux stratégies. En fait, quand on applique l'imputation par la moyenne et que l'on calcule la moyenne empirique, cela revient à calculer une moyenne avec plus de valeurs. A priori, cette stratégie est donc plus pertinente.

Dans la stratégie où l'on supprime les lignes incomplètes, on supprime des lignes entières, dont des valeurs observées.
Pour estimer la moyenne de chaque variable, c'est plus naturel, tout en restant simple, d'utiliser toutes les valeurs observées, en calculant une moyenne par variable et en ignorant les valeurs manquantes de cette variable (ce qui est fait dans la méthode d'imputation par la moyenne).

Notons cependant que beaucoup de méthodes d'apprentissage utilisent des blocs de variables pour tenir compte des liens entre les elles. Dans ce cas, on ne peut pas appliquer l'algorithme par variable. C'est pour cette raison que la stratégie où les lignes incomplètes sont supprimés est largement répandue.

# Score de référence

## Question 8

Dans la comparaison précédente, vous n'avez pas de score de référence.

Une solution consiste à générer de nouvelles valeurs manquantes pour obtenir un jeu de données *amputé*, appliquer les deux méthodes sur ce jeu de données et les comparer aux quantités qu'on aura obtenu sur le jeu de données initial, qui seront considérées comme les scores de référence.

Pourquoi selon vous est-ce intéressant de connaître les patterns les plus fréquents dans le jeu de données ? À quoi correspondent les `0`et les `1` du tableau ci-dessous `frequent.pattern` ?

In [ ]:
# Patterns les plus fréquents
a <- aggr(ozone, plot = FALSE)
idx.frequent.pattern <- sort(a$percent, index.return = TRUE, decreasing = TRUE)
frequent.pattern <- matrix(0,nrow=5,ncol=dim(ozone)[2])
for (i in 1:5) {
  frequent.pattern[i,] <- a$tabcomb[idx.frequent.pattern$ix[i], ]
}
colnames(frequent.pattern)=colnames(ozone)

In [ ]:
frequent.pattern

### Correction

On peut générer de nouvelles valeurs manquantes sur les lignes complètes avec les patterns les plus fréquents dans le jeu de données pour que ces nouvelles valeurs manquantes soient les plus réalistes possibles.

Le tableau `frequent.pattern` affiche les patterns les plus fréquent. Par exemple, pour la colonne `T12`, sa valeur est observée pour le premier pattern (première ligne) et manquante pour le cinquième pattern (cinquième ligne). Un `0` indique que la valeur de la variable est observée pour ce pattern, un `1` indique au contraire qu'elle est manquante.

## Question 9

Le pattern le plus fréquent est une ligne complète (seulement des 0 dans `frequent.pattern`). On génère donc les valeurs manquantes selon les 4 patterns les plus fréquents donnés à la question précédente (sauf le premier).

Est-ce que toutes les variables ont de nouvelles valeurs manquantes ? Comment interprétez-vous le graphique de droite ?

In [ ]:
frequent.pattern <- frequent.pattern[-1,]

In [ ]:
pattern.indices <- sample(1:nrow(frequent.pattern), size = n.full.rows, replace = TRUE) # on sélectionne les patterns
ozone.addNA <- ozone

c <- 0

for (i in 1:dim(ozone)[1]) {
  if (sum(is.na(ozone[i,]))==0){
    c <- c+1
    pattern <- frequent.pattern[pattern.indices[c], ]
    ozone.addNA[i, pattern == 1] <- NA
  }
}
paste("Lignes complètes dans le jeu de données initial:", dim(ozone.full)[1])
paste("Nombre de lignes où des nouvelles NA sont introduites:", c)

In [ ]:
a <- aggr(ozone.addNA, plot = FALSE)
par(bg = "white") # définit le fond en blanc
plot(a, numbers = TRUE, prop = FALSE)

### Correction

Seules les variables `T12`, `T15`, `Ne15` et `Vx15` ont de nouvelles valeurs manquantes (on le voit dans la matrice `frequence.pattern`).
Dans le graphique de droite, on observe qu'il n'y a plus aucune ligne complète dans le jeu de données amputé. C'est normal, car des valeurs manquantes ont été générées selon les patterns les plus fréquents, qui sont les 4 dernières lignes du graphique de droite.

## Question 11

Le code suivant compare la méthode d'imputation par la moyenne obtenue sur le jeu de données amputé au score de référence obtenu sur le jeu de données initial.
Les biais pour l'estimation de la moyenne empirique et de la variance empirique sont calculés par variable.

Qu'observez-vous ?

In [ ]:
# Scores de référence sur le jeu de données initial
score.ref.mean <- apply(ozone, 2, mean, na.rm = TRUE)
score.ref.var <- apply(ozone, 2, var, na.rm = TRUE)

In [ ]:
# Résultats avec l'imputation par la moyenne obtenue sur le jeu de données amputé
ozone.addNA.mean <- impute.mean(ozone.addNA)
imputation.mean.mean <- apply(ozone.addNA.mean, 2, mean)
imputation.mean.var <- apply(ozone.addNA.mean, 2, var)

In [ ]:
biais.mean <- (imputation.mean.mean-score.ref.mean)
biais.var <- (imputation.mean.var-score.ref.var)

In [ ]:
biais.mean[c(3, 4, 7, 10)] # on affiche les biais que sur les variables où il y a de nouvelles valeurs manquantes

In [ ]:
biais.var[c(3, 4, 7, 10)] # on affiche les biais que sur les variables où il y a de nouvelles valeurs manquantes

### Correction

L'estimation de la variance est largement biaisée, alors que l'estimation de la moyenne a des biais beaucoup plus faibles. En fait, dans le cas MCAR (lorsque le fait que les données manquent ne dépend pas des valeurs des données elles-mêmes), l'imputation par la moyenne permet d'estimer la moyenne empirique sans induire de biais supplémentaire dû à la présence des valeurs manquantes. Par contre, l'estimation de la variance est biaisée.

## Question 12

Le graphique ci-dessous représente le nuage de points des valeurs (observées et imputées).

Que pouvez-vous en conclure sur la méthode d'imputation par la moyenne ?

In [ ]:
mask <- is.na(ozone.addNA)

plot(ozone.addNA$T12, ozone.addNA$T15, col = "#d1e5f0", xlab = "T12", ylab = "T15")
points(ozone.addNA.mean$T12[mask[, 3] == 1], ozone.addNA.mean$T15[mask[, 3] == 1], col = "#2194ac")
points(ozone.addNA.mean$T12[mask[, 4] == 1], ozone.addNA.mean$T15[mask[, 4] == 1], col = "#2138ac")
points(ozone.addNA.mean$T12[mask[, 3] == 1 & mask[, 4] == 1], ozone.addNA.mean$T15[mask[, 3] == 1 & mask[, 4] == 1], col = "#ac6721")

legend("bottomright",
  legend = c("Complet", "X0 imputé", "X1 imputé", "X0,X1 imputés"),
  col = c("#d1e5f0", "#2194ac", "#2138ac", "#ac6721"),
  pch = 16,
  bty = "n"
)

### Correction

La distribution des données est déformée et le lien entre les variables n'est pas respecté si on utillise la méthode d'imputation par la moyenne.